# Урок 2. Парсинг HTML. BeautifulSoup, MongoDB (Урок 3 ниже)

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы)

Получившийся список должен содержать в себе минимум:

* Наименование вакансии.
* Предлагаемую зарплату (отдельно минимальную и максимальную).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). 
Структура должна быть одинаковая для вакансий с обоих сайтов. 

Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

### Import libraries

In [178]:
import requests
import re
import pandas as pd
import json

from bs4 import BeautifulSoup as bs

### Define helper functions

In [179]:
def getCompensationCurrency(str):
  return str.split()[-1]

def getMinCompensation(str):
  numbers = re.findall(r'\d+', str.replace(' ', ''))
  if 'до' in str:
    return None
  else:
    return int(numbers[-1]) if 'от' in str else int(numbers[0])

def getMaxCompensation(str):
  numbers = re.findall(r'\d+', str.replace(' ', ''))
  if 'от' in str:
    return None
  else:
    return int(numbers[0]) if 'до' in str else int(numbers[-1])

In [180]:
def parseJob(job):
  title = job.find('a', {'class': 'bloko-link'}).getText()
  link = job.find('a', {'class': 'bloko-link'}).get('href')

  compensation = job.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
  compensationText = None if compensation is None else compensation.getText().replace(u'\u202f', ' ')
  
  minCompensation = None if compensationText is None else getMinCompensation(compensationText)
  maxCompensation = None if compensationText is None else getMaxCompensation(compensationText)
  compensationCurrency = None if compensationText is None else getCompensationCurrency(compensationText)

  return {
      'title': title,
      'link': link,
      'minCompensation': minCompensation,
      'maxCompensation': maxCompensation,
      'compensationCurrency': compensationCurrency
  }

### Get the search request

In [181]:
searchRequest = input('Enter a search request: ')

Enter a search request: scala js


### Make a request

In [182]:
fullJobsList = []

In [183]:
baseUrl = 'https://hh.ru'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0'
}

params = {
    'area': '1',
    'fromSearchLine': 'true',
    'st': 'searchVacancy',
    'text': searchRequest
}

url = baseUrl + '/search/vacancy'

while True:
  response = requests.get(url, params=params, headers=headers)
  soup = bs(response.text, 'html.parser')

  jobs = soup.find_all('div', {'class': 'vacancy-serp-item'})

  parsedJobs = list(map(lambda x: parseJob(x), jobs))
  fullJobsList.extend(parsedJobs)

  nextButton = soup.find('a', {'data-qa': 'pager-next'})
  url = None if nextButton is None else baseUrl + nextButton.get('href')

  params = None
  
  if url is None:
    break

### Convert results to Pandas DataFrame

In [184]:
df = pd.DataFrame(fullJobsList)
df

,title,link,minCompensation,maxCompensation,compensationCurrency
0,Scala разработчик,https://hh.ru/vacancy/46861396?from=vacancy_se...,90000.0,110000.0,руб.
1,Front-end разработчик (React),https://hh.ru/vacancy/46744457?from=vacancy_se...,NaN,NaN,None
2,Back-End Developer,https://hh.ru/vacancy/46138566?from=vacancy_se...,2000.0,NaN,USD
3,Scala-разработчик/Backend-разработчик (удаленно),https://hh.ru/vacancy/46389403?from=vacancy_se...,NaN,NaN,None
4,Junior Scala-разработчик (удаленно),https://hh.ru/vacancy/45250274?from=vacancy_se...,NaN,NaN,None
5,Front-end junior+ разработчик (React),https://hh.ru/vacancy/46596080?from=vacancy_se...,70000.0,NaN,руб.
6,Kotlin Backend Developer,https://hh.ru/vacancy/41711824?from=vacancy_se...,NaN,NaN,None
7,PHP developer (middle),https://hh.ru/vacancy/45279715?from=vacancy_se...,NaN,NaN,None
8,"Senior Software Engineer, Database Engineering...",https://hh.ru/vacancy/46377490?from=vacancy_se...,6000.0,NaN,USD
9,Senior Java developer - Rates trade processing,https://hh.ru/vacancy/46716389?from=vacancy_se...,NaN,NaN,None


### Save results as JSON

In [185]:
with open('hh.json', 'w') as fout:
    json.dump(fullJobsList , fout)

# Урок 3. Системы управления базами данных MongoDB и SQLite в Python

### Install dependencies (Restart runtime after the first installation)

In [186]:
! pip3 install pymongo[srv]

### Import libraries

In [187]:
from pymongo import MongoClient 

### Connect to the data base

In [189]:
usr = 'db_user'
pwd = 'mOS9LygLPkiXsliW'
dbname = 'ds_course'

url = 'mongodb+srv://' + usr + ':' + pwd + '@myfreecluster.sns97.mongodb.net/' + dbname + '?retryWrites=true&w=majority'

client = MongoClient(url)

In [190]:
db = client[dbname]
jobs_list_collection = db.job_list

### Insert data

Use **insert_many** if collectino is empty, otherwise use **update_one** with *upsert=True*.

In [194]:
if jobs_list_collection.count_documents({}) == 0:
  jobs_list_collection.insert_many(fullJobsList)
else:
  for job in fullJobsList:
    query = {
        'link': job['link']
    }

    update = {
        '$set': { 
            'title': job['title'],
            'link': job['link'],
            'minCompensation': job['minCompensation'],
            'maxCompensation': job['maxCompensation'],
            'compensationCurrency': job['compensationCurrency']
        }
    }

    jobs_list_collection.update_one(query, update, upsert=True)

### Make a filtering request

In [192]:
minimumSalary = input('Enter minimum salary: ')

Enter minimum salary: 100000


In [193]:
minimumSalaryJobs = jobs_list_collection.find({'minCompensation': {'$gt': int(minimumSalary)}})
list(minimumSalaryJobs)

[{'_id': ObjectId('61108180b6cd48055041f330'),
  'compensationCurrency': 'руб.',
  'link': 'https://hh.ru/vacancy/46149829?from=vacancy_search_list&query=scala%20js',
  'maxCompensation': None,
  'minCompensation': 150000,
  'title': 'QA engineer (mobile)'},
 {'_id': ObjectId('61108180b6cd48055041f331'),
  'compensationCurrency': 'руб.',
  'link': 'https://hh.ru/vacancy/46816670?from=vacancy_search_list&query=scala%20js',
  'maxCompensation': None,
  'minCompensation': 150000,
  'title': 'Senior PHP developer'},
 {'_id': ObjectId('61108180b6cd48055041f336'),
  'compensationCurrency': 'руб.',
  'link': 'https://hh.ru/vacancy/46823836?from=vacancy_search_list&query=scala%20js',
  'maxCompensation': 160000,
  'minCompensation': 120000,
  'title': 'Data Engineer Middle'}]